# CS676 Final Project

### Aayushi Verma & Kelsey Woods

In this project, we have attempted Question 23.1 of Exercise 23.5 in Professor Cha's MIDAS textbook (Chapter 23, VII-17).

There are two parts to this project:
1. Implementing the hierarchal agglomerative clustering algorithm using the numpy package, and allowing the user to choose one of three distance measures (L1, L2 and LP).
2. An attempt at implementing the hierarchal agglomerative clustering algorithm using the pandas package, and allowing the user to choose one of three distance measures (L1, L2 and LP), and different linkage types (complete).

In [1]:
# importing packages for data cleaning, visualization, and EDA
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame(
    {
        'x':[4,8,15,24,24],
        'y':[4,4,8,4,12]
    }
)

## Section One: Algorithm implementation with numpy

In [3]:
def distance_matrix_creation(data):
    n = data.shape[0]
    distances = np.zeros((n, n))
    np.fill_diagonal(distances, np.inf)
    return distances, n

def np_l1(x,y):
      return np.abs((x[0] - y[0])) + np.abs((x[1] - y[1]))
def np_l2(x,y):
      return np.sqrt(((x[0] - y[0]) ** 2) + ((x[1] - y[1]) ** 2))
def np_lp(x,y,p):
      return ((np.abs(x[0] - y[0]) ** p) + (np.abs(x[1] - y[1]) ** p)) ** p

def distance_calculations(data, n, distances, distance_measure='l2', p=None):
    for i in range(n):
        for j in range(i + 1, n):
            if distance_measure == 'l1':
                distances[i][j] = np_l1(data[i], data[j])
                distances[j][i] = distances[i][j]
            elif distance_measure == 'l2':
                distances[i][j] = np_l2(data[i], data[j])
                distances[j][i] = distances[i][j]
            else:
                distances[i][j] = np_lp(data[i], data[j], p=1)
                distances[j][i] = distances[i][j]
    return distances

def cluster_merging(distances, clusters, num_clusters):
    while len(clusters) > num_clusters:
        # Find the two closest clusters
        min_distance = np.inf
        for i in range(len(clusters)):
            for j in range(i + 1, len(clusters)):
                distance = 0
                for p in clusters[i]:
                    for q in clusters[j]:
                        distance += distances[p][q]
                distance /= len(clusters[i]) * len(clusters[j])
                if distance < min_distance:
                    min_distance = distance
                    merge_clusters = (i, j)

    # Merge the two closest clusters
    clusters[merge_clusters[0]] += clusters[merge_clusters[1]]
    del clusters[merge_clusters[1]]

    return clusters

def cluster_assignment(clusters, num_clusters):
    labels = np.zeros(n, dtype=int)
    for i in range(num_clusters):
        for j in clusters[i]:
            labels[j] = i
    return labels

def hierarchal_clustering_agglomerative(data, num_clusters, distance_measure):
    distances, n = distance_matrix_creation(data)
    distances = distance_calculations(data, n, distances, distance_measure, p=None)
    clusters = [[i] for i in range(n)]
    clusters = cluster_merging(distances, clusters, num_clusters)
    labels = cluster_assignment(clusters, num_clusters)
    return labels

In [4]:
data = df.to_numpy()

In [5]:
labels = hierarchal_clustering_agglomerative(data, num_clusters=3, distance_measure='l1')
labels

## Section Two: Attempt at algorithm implementation with pandas

In [ ]:
def min_value_location(df):
    row_idx_of_min_values_in_df = df.idxmin(axis=0,numeric_only=True).values.tolist()
    col_idx_of_min_values_in_df = df.idxmin(axis=1,numeric_only=True).values.tolist()
    lst_of_min_values_in_df = df.min(numeric_only=True).values.tolist()
    min_value = min(lst_of_min_values_in_df)
    idx_of_min_value = lst_of_min_values_in_df.index(min_value)
    idx_of_min_row_in_df = row_idx_of_min_values_in_df[idx_of_min_value]
    idx_of_min_col_in_df = col_idx_of_min_values_in_df[idx_of_min_value]
    new_cluster_location = [idx_of_min_row_in_df,int(idx_of_min_col_in_df)]
    return new_cluster_location, min_value

def l1(given_point,x,y):
    # Function to determine Manhattan distance, L1.
    d_l1 = []
    d = len(x) # assumes len(x) = len(y)
    for i in range(d):
        d_sum = (given_point[0] - x[i]) + (given_point[1] - y[i])
        d_l1.append(d_sum)
    return d_l1

def l2(given_point,x,y):
    # Function to determine Euclidean distance, L2.
    d_l2 = []
    d = len(x) # assumes len(x) = len(y)
    for i in range(d):
        d_sum = np.sqrt(((given_point[0] - x[i]) ** 2) + ((given_point[1] - y[i]) ** 2))
        d_l2.append(d_sum)
    return d_l2

def l3(given_point, x, y, p):
    # Function to determine Minkowski distance, L3.
    d_l3 = []
    d = len(x) # assumes len(x) = len(y)
    for i in range(d):
        d_sum = (((given_point[0] - x[i]) ** p) + ((given_point[1] - y[i]) ** p)) ** (1 / p)
        d_l3.append(d_sum)
    return d_l3

# step 1: initialize df
def initialize_cluster(df, distance_measure='l2'):
    df2 = []
    for i in range(len(df)):
        given_point = df.iloc[i]

        if distance_measure == 'l2':
            df2.append(l2(given_point,df['x'],df['y']))
        else:
            # dummy clause for now
            df2.append(l2(given_point,df['x'],df['y']))

    df3 = pd.DataFrame(df2)
    df3 = df3.drop(0, axis=1)

    blank_value = pd.NA
    for i in range(len(df3)):
        df3.loc[i:,i] = blank_value
    df3.drop(4,axis=0,inplace=True) # generatlize this [-1]
    # df3.drop(0, axis=1,inplace=True)
    return df3

def complete_linkage(df1):
    return df1.max()

def old_cluster(df1, new_cluster):
    df2 = df1.loc[[new_cluster[0],new_cluster[1]],:]
    df2.drop(new_cluster[0],axis=1,inplace=True)

    df1.drop([new_cluster[0]+1,new_cluster[1]+1],axis=1,inplace=True)
    df1.drop(new_cluster,axis=0,inplace=True)
    return df1, df2

def modifying_rows_cols(df1, df2):
    if 0 in df1.columns:
        df1.drop(0, axis=1,inplace=True)
    df1[df1.columns[-1]+1] = pd.Series()
    df1 = pd.concat(
        [df1, pd.Series(index=[df1.index[-1]+1], data=pd.NA).to_frame(name=df1.index[-1]+1).T], 
        axis=0, ignore_index=False
        )
    return df1, df2

def linkage_insertion(df1, df2, linkage_type):
    if linkage_type == 'complete':
        df1[df1.columns[-1]] = complete_linkage(df2)
    else:
        # dummy clause for now
        df1[df1.columns[-1]] = complete_linkage(df2)
    return df1

def clustering_iteration(df1, linkage_type='complete'):
    new_cluster, min_value = min_value_location(df1)
    df1, df2 = old_cluster(df1, new_cluster)
    df1, df2 = modifying_rows_cols(df1, df2)
    df1 = linkage_insertion(df1, df2, linkage_type)

    return df1

In [ ]:
# Iteration 1
df3 = initialize_cluster(df, distance_measure='l2')
df3

In [ ]:
# Iteration 2 - giving incorrect results
df7 = clustering_iteration(df3, linkage_type='complete')
df7